In [1]:
!pip install transformers[torch] datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.5 MB/s eta 0:00:00


In [4]:
import torch.nn as nn
import numpy as np

from datasets import load_dataset, load_metric
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from transformers import TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support

# Define dataset
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

# Define model and tokenizer
checkpoint = "distilbert-base-cased"
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
model = DistilBertForSequenceClassification.from_pretrained(checkpoint, num_labels=4)

# Function to tokenize examples
def tokenize_function(examples):
    inputs = tokenizer(examples["sentence"], padding="max_length", truncation=True)
    inputs["labels"] = examples["tier"]

    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Accuracy
    acc = metric.compute(predictions=predictions, references=labels)

    # Precision, Recall, F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        'accuracy': acc['accuracy'],
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Create training arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=10,
    logging_dir="logs",
    logging_steps=500,
)

# Initialize trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

Map:   0%|          | 0/4344 [00:00<?, ? examples/s]

Map:   0%|          | 0/1086 [00:00<?, ? examples/s]

<ipython-input-4-2e468f4d5c5c>:26: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.520100,0.284461,0.907919,0.907843,0.907919,0.907736
2,0.237500,0.314124,0.905157,0.908509,0.905157,0.905621
3,0.166600,0.419057,0.911602,0.914727,0.911602,0.912129
4,0.117300,0.384763,0.924494,0.924642,0.924494,0.924473
5,0.078100,0.481375,0.911602,0.913907,0.911602,0.911920
6,0.089400,0.527616,0.913444,0.913748,0.913444,0.913504
7,0.064200,0.482124,0.921731,0.922458,0.921731,0.921893
8,0.053200,0.570912,0.919890,0.921135,0.919890,0.920107
9,0.053600,0.585588,0.919890,0.921984,0.919890,0.920154
10,0.028800,0.588948,0.919890,0.921211,0.919890,0.920084


TrainOutput(global_step=5430, training_loss=0.1324806687581605, metrics={'train_runtime': 817.6797, 'train_samples_per_second': 53.126, 'train_steps_per_second': 6.641, 'total_flos': 5754589040148480.0, 'train_loss': 0.1324806687581605, 'epoch': 10.0})